In [24]:
import pandas as pd
import numpy as np
from prediction.functions import aggregate_prediction_results

In [28]:
models = ['logit', 'ridge', 'lasso', 'enet', 'random_forest', 'lgb', 'nn3']
models2 = ['LOGIT', 'RIDGE', 'LASSO', 'ENET', 'RF', 'GB', 'NN3']
fs_methods = ['all', 'sfi', 'mdi', 'mda', 'granger', 'huang', 'IAMB', 'MMMB']
fs_methods2 = ['ALL', 'SFI', 'MDI', 'MDA', 'GRANGER', 'HUANG', 'IAMB', 'MMMB']
tickers = ['SPX Index', 'CCMP Index','RTY Index', 'SPX Basic Materials',
           'SPX Communications', 'SPX Consumer Cyclical',
           'SPX Consumer Non cyclical', 'SPX Energy', 'SPX Financial',
           'SPX Industrial', 'SPX Technology', 'SPX Utilities']
benchmark_name = 'return_direction'
target_name = 'target_return'
metric = "auc"
oos_start_date = '2012-07-03'
oos_end_date = '2020-12-31'

In [36]:
melted_predictions_df, melted_benchmark_df, _ = aggregate_prediction_results(prediction_models=models,
                                                                             fs_models=[fs_methods[0]],
                                                                             evaluation_start_date='2005-01-03',
                                                                             evaluation_end_date=oos_end_date,
                                                                             ticker_names=[tickers[0]],
                                                                             metric_name=metric,
                                                                             tag='is',
                                                                             benchmark_name=benchmark_name)

# turn predictions into a pivot table
predictions_df = melted_predictions_df.pivot_table(index=['date'], columns=['model'], values=['value'])
predictions_df.columns = predictions_df.columns.droplevel()

# turn benchmark into a pivot table
benchmark_df = melted_benchmark_df.pivot_table(index=['date'], columns=['model'], values=['value'])
benchmark_df.columns = benchmark_df.columns.droplevel()

df = pd.concat([benchmark_df, predictions_df], axis=1)
df.index = pd.to_datetime(df.index)
df.rename(columns={tickers[0]: target_name}, inplace=True)

years = df.index.map(lambda x: x.year)
years = range(np.min(years), np.max(years))
features = sorted(df.drop(target_name, 1).columns.to_list())
df = df[features + [target_name]]

y = years[0]

train_ys = df.loc[:str(y)]
test_ys = df.loc[str(y + 1)]

target_train = train_ys[target_name]
train_ys = train_ys.drop(target_name, axis=1)
target_test = test_ys[target_name]
test_ys = test_ys.drop(target_name, axis=1)

/Users/danieloliveira/Documents/Daniel/codes/causal_feature/src/prediction/functions.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  benchmark_df['model'] = benchmark_df['ticker']
/Users/danieloliveira/Documents/Daniel/codes/causal_feature/src/prediction/functions.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  benchmark_df['fs'] = 'raw'


## 1. Combinação de previsões sem estimação

### a) Média

### b) Mediana 



## 2. Combinação de previsões com estimação

### a) Bates e Granger (1969)

In [152]:
m = train_ys.shape[1]
l_m = np.ones((m, 1))

comb_dict = {}
for i, row in train_ys.reset_index().iterrows():
    y_pred = train_ys.to_numpy()[i]
    y_target = target_train.values[i]

    e = l_m.dot(y_target) - y_pred.reshape(-1,1)
    cov_e = e.dot(e.transpose())
    
    if np.linalg.det(cov_e) != 0:
        inv_cov_e = np.linalg.inv(cov_e)
    else:
        inv_cov_e = np.linalg.pinv(cov_e)

    w_star = np.linalg.inv(l_m.transpose().dot(inv_cov_e).dot(l_m)) * inv_cov_e.dot(l_m)

    if i+1 < train_ys.reset_index().index[-1]:
        y_pred_t1 = train_ys.to_numpy()[i+1]
        comb_pred = y_pred_t1.dot(w_star)
        
        comb_dict[i+1] = {'date': train_ys.iloc[i+1].name,
                          'return_direction': target_train.iloc[i+1],
                          'prediction': comb_pred[0]}
        

comb_predictions_out = pd.DataFrame(comb_dict).T

### b) Rede Neural

### c) peLASSO